In [24]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.8f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [25]:
events_training = pd.read_csv('training.csv')
events_test = pd.read_csv('test.csv')
training = pd.read_csv('labels_training_set.csv')
test =  pd.read_csv('trocafone_kaggle_test.csv')

In [26]:
events_training.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00000000,iPhone 5s,Muito Bom,32GB,Cinza espacial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00000000,iPhone 5s,Muito Bom,64GB,Prateado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00000000",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [27]:
events_test.head()

,Unnamed: 0,timestamp,event,person,url,sku,model,condition,storage,color,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,0,2018-05-18 00:11:59,viewed product,4886f805,NaN,"9,288.00000000",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-05-18 00:30:30,viewed product,4886f805,NaN,"9,288.00000000",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,nan,NaN,NaN,NaN,NaN,...,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-05-18 00:11:56,checkout,4886f805,NaN,"9,288.00000000",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-05-18 00:11:35,viewed product,4886f805,NaN,"9,287.00000000",Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# PRIMERO VAMOS A TRABAJAR CON EL DATAFRAME DE ENTRENAMIENTO

# Hacemos un feature engineering que es un one encoding modificaco. Se dice modificado porque en cada casillero ponemos la cantidad de cada. Uno de los eventos que hizo cada usuario.

In [28]:
sum_enconding=events_training.groupby(['person','event']).agg({'event':'count'}).unstack().fillna(0)
sum_enconding.columns=['ad campaign hit','brand listing','checkout','conversion','generic listing'\
                       ,'lead','search engine hit','searched products','staticpage','viewed product','visited site']

In [29]:
sum_enconding=sum_enconding.reset_index() 
sum_enconding.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0.00000000,0.00000000,3.00000000,0.00000000,1.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2.00000000
1,000c79fe,1.00000000,0.00000000,1.00000000,0.00000000,1.00000000,0.00000000,1.00000000,9.00000000,0.00000000,3.00000000,1.00000000
2,001802e4,5.00000000,0.00000000,1.00000000,0.00000000,4.00000000,0.00000000,0.00000000,4.00000000,0.00000000,4.00000000,1.00000000
3,0019e639,29.00000000,165.00000000,15.00000000,2.00000000,28.00000000,0.00000000,13.00000000,11.00000000,0.00000000,189.00000000,19.00000000
4,001b0bf9,0.00000000,1.00000000,2.00000000,1.00000000,1.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2.00000000,0.00000000


# Aca le vamos a asignar a cada usuario cual fue el evento que mas hicieron y cuantas veces lo hicieron, y cual fue el evento que menos realizaron y cuantas veces hicieron el mismo.

In [30]:
datos_uno = events_training.loc[:,['person','event']]

In [31]:
cantidad_eventos = datos_uno.groupby('person').count().reset_index()
cantidad_eventos.columns=['person','event_count']
cantidad_eventos.head()

,person,event_count
0,0008ed71,6
1,000c79fe,17
2,001802e4,19
3,0019e639,471
4,001b0bf9,7


In [32]:
datos_dos = events_training.loc[:,['person','event','timestamp']]
datos_dos = datos_dos.groupby(['person','event']).count().reset_index()
datos_dos.columns=['person','event','cant_per_event']
datos_dos.head()

,person,event,cant_per_event
0,0008ed71,checkout,3
1,0008ed71,generic listing,1
2,0008ed71,visited site,2
3,000c79fe,ad campaign hit,1
4,000c79fe,checkout,1


In [34]:
maximo_evento = datos_dos.groupby('person').apply(lambda x: x.nlargest(1,'cant_per_event')).reset_index(drop=True)
maximo_evento.columns=['person','most_frequent_event','count_most_frequent_event']
maximo_evento.head()

,person,most_frequent_event,count_most_frequent_event
0,0008ed71,checkout,3
1,000c79fe,searched products,9
2,001802e4,ad campaign hit,5
3,0019e639,viewed product,189
4,001b0bf9,checkout,2


In [35]:
minimo_evento = datos_dos.groupby('person').apply(lambda x: x.nsmallest(1,'cant_per_event')).reset_index(drop=True)
minimo_evento.columns=['person','less_frequent_event','count_less_frequent_event']
minimo_evento.head()

,person,less_frequent_event,count_less_frequent_event
0,0008ed71,generic listing,1
1,000c79fe,ad campaign hit,1
2,001802e4,checkout,1
3,0019e639,conversion,2
4,001b0bf9,brand listing,1


# Agregamos un feature mas que es el promedio de la cantidad de cada evento que realizo el usuario.

In [36]:
media_evento = datos_dos.loc[:,['person','cant_per_event']]\
                .groupby('person').agg(lambda x: x.mean()).reset_index()
media_evento.columns=['person','mean_per_event']
media_evento.head(5)

,person,mean_per_event
0,0008ed71,2.00000000
1,000c79fe,2.42857143
2,001802e4,3.16666667
3,0019e639,52.33333333
4,001b0bf9,1.40000000


# Ahora agregamos los features en un solo dataframe.

In [37]:
valores = training.merge(maximo_evento,on='person',how='inner').merge(minimo_evento,on='person',how='inner')\
        .merge(sum_enconding,on='person',how='inner').merge(media_evento,on='person',how='inner')

In [38]:
valores.head()

,person,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,mean_per_event
0,0566e9c1,0,viewed product,23,checkout,1,6.00000000,3.00000000,1.00000000,1.00000000,15.00000000,0.00000000,1.00000000,0.00000000,1.00000000,23.00000000,17.00000000,7.55555556
1,6ec7ee77,0,ad campaign hit,1,ad campaign hit,1,1.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,1.00000000,1.00000000
2,abe7a2fb,0,viewed product,31,checkout,1,9.00000000,14.00000000,1.00000000,0.00000000,9.00000000,0.00000000,4.00000000,6.00000000,0.00000000,31.00000000,22.00000000,12.00000000
3,34728364,0,viewed product,24,search engine hit,1,0.00000000,5.00000000,0.00000000,0.00000000,3.00000000,0.00000000,1.00000000,0.00000000,0.00000000,24.00000000,4.00000000,7.40000000
4,87ed62de,0,viewed product,9,visited site,1,5.00000000,0.00000000,2.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,9.00000000,1.00000000,4.25000000


# AHORA VOY A REPETIR TODO ESTO PARA EL DATAFRAME DE TESTEO

# Agrego el feature de one hot encodig modificado que hicimos antes.

In [39]:
sum_enconding_test=events_test.groupby(['person','event']).agg({'event':'count'}).unstack().fillna(0)
sum_enconding_test.columns=['ad campaign hit','brand listing','checkout','conversion','generic listing','lead','search engine hit','searched products','staticpage','viewed product','visited site']
sum_enconding_test=sum_enconding_test.reset_index() 
sum_enconding_test.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,00091926,15.00000000,25.00000000,2.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,372.00000000,34.00000000
1,00091a7a,1.00000000,5.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,3.00000000,1.00000000
2,000ba417,1.00000000,24.00000000,6.00000000,1.00000000,14.00000000,0.00000000,1.00000000,0.00000000,0.00000000,153.00000000,6.00000000
3,000e4d9e,19.00000000,17.00000000,1.00000000,0.00000000,17.00000000,0.00000000,5.00000000,0.00000000,0.00000000,339.00000000,13.00000000
4,000e619d,6.00000000,11.00000000,1.00000000,0.00000000,8.00000000,0.00000000,3.00000000,6.00000000,0.00000000,28.00000000,5.00000000


# Ahora calculo el evento maximo de cada usuario y su cantidad,y el evento minimo y su cantidad para cada usuario

In [40]:
datos_dos_test = events_test.loc[:,['person','event','timestamp']]
datos_dos_test = datos_dos_test.groupby(['person','event']).count().reset_index()
datos_dos_test.columns=['person','event','cant_per_event']
datos_dos_test.head()

,person,event,cant_per_event
0,00091926,ad campaign hit,15
1,00091926,brand listing,25
2,00091926,checkout,2
3,00091926,viewed product,372
4,00091926,visited site,34


In [41]:
maximo_evento_test = datos_dos_test.groupby('person').apply(lambda x: x.nlargest(1,'cant_per_event'))\
                .reset_index(drop=True)
maximo_evento_test.columns=['person','most_frequent_event','count_most_frequent_event']
maximo_evento_test.head()

,person,most_frequent_event,count_most_frequent_event
0,00091926,viewed product,372
1,00091a7a,brand listing,5
2,000ba417,viewed product,153
3,000e4d9e,viewed product,339
4,000e619d,viewed product,28


In [42]:
minimo_evento_test = datos_dos_test.groupby('person').apply(lambda x: x.nsmallest(1,'cant_per_event'))\
        .reset_index(drop=True)
minimo_evento_test.columns=['person','less_frequent_event','count_less_frequent_event']
minimo_evento_test.head()

,person,less_frequent_event,count_less_frequent_event
0,00091926,checkout,2
1,00091a7a,ad campaign hit,1
2,000ba417,ad campaign hit,1
3,000e4d9e,checkout,1
4,000e619d,checkout,1


# Ahora calcula la media de la cantidad de eventos para cada usuario.

In [43]:
media_evento_test = datos_dos_test.loc[:,['person','cant_per_event']]\
                .groupby('person').agg(lambda x: x.mean()).reset_index()
media_evento_test.columns=['person','mean_per_event']


In [44]:
media_evento_test.head()

,person,mean_per_event
0,00091926,89.60000000
1,00091a7a,2.50000000
2,000ba417,25.75000000
3,000e4d9e,58.71428571
4,000e619d,8.50000000


# Junto todos los features en un solo dataframe para cada usuario

In [45]:
valores_test = test.merge(maximo_evento_test,on='person',how='inner').merge(minimo_evento_test,on='person',how='inner')\
                .merge(media_evento_test,on='person',how='inner').merge(sum_enconding_test,on='person',how='inner')

In [46]:
valores_test.head()

,person,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,mean_per_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,4886f805,viewed product,4,checkout,1,1.50000000,0.00000000,0.00000000,1.00000000,0.00000000,1.00000000,0.00000000,1.00000000,1.00000000,0.00000000,4.00000000,1.00000000
1,0297fc1e,viewed product,404,lead,1,70.87500000,29.00000000,4.00000000,7.00000000,0.00000000,21.00000000,1.00000000,0.00000000,6.00000000,0.00000000,404.00000000,95.00000000
2,2d681dd8,viewed product,13,ad campaign hit,1,3.25000000,1.00000000,5.00000000,1.00000000,0.00000000,1.00000000,0.00000000,2.00000000,1.00000000,0.00000000,13.00000000,2.00000000
3,cccea85e,viewed product,739,checkout,1,92.88888889,15.00000000,7.00000000,1.00000000,0.00000000,20.00000000,0.00000000,26.00000000,1.00000000,5.00000000,739.00000000,22.00000000
4,4c8a8b93,viewed product,177,checkout,2,32.12500000,14.00000000,8.00000000,2.00000000,0.00000000,14.00000000,0.00000000,13.00000000,9.00000000,0.00000000,177.00000000,20.00000000


# Guardo los dataframes en dos archivos csv que seran usados en otros programas.

In [47]:
valores.to_csv('train_paso_1.csv',index=False)
valores_test.to_csv('test_paso_1.csv',index=False)